In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
convlayer = torch.nn.Conv1d(1,256,5)

In [3]:
x = torch.randint(1,10,(1,20)).type(torch.float)

In [4]:
print(x)

tensor([[1., 6., 1., 3., 6., 9., 4., 7., 3., 9., 7., 5., 3., 9., 3., 6., 9., 5.,
         4., 7.]])


In [5]:
print(x.unsqueeze(0).shape)

torch.Size([1, 1, 20])


In [6]:
class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True):

        super(CausalConv1d, self).__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=0,
            dilation=dilation,
            groups=groups,
            bias=bias)
        
        self.__padding = (kernel_size - 1) * dilation
        
    def forward(self, input):
        return super(CausalConv1d, self).forward(F.pad(input, (self.__padding, 0)))

In [7]:
kernel_size = 3
in_channels=1
out_channels=1
dilation = 1

In [8]:
convlayer = CausalConv1d(in_channels,out_channels,kernel_size,dilation=dilation)

In [9]:
print(convlayer(x.unsqueeze(0)).shape)
print(convlayer(x.unsqueeze(0)))

torch.Size([1, 1, 20])
tensor([[[-0.0975, -0.2860,  2.2729, -1.9973,  0.2712,  0.5473,  1.5007,
          -2.1990,  1.4213, -2.2102,  2.1639, -0.5824, -0.5335, -1.5114,
           2.6773, -2.5240,  0.5473,  1.3724, -1.3607, -0.8014]]],
       grad_fn=<SqueezeBackward1>)


In [10]:
__padding = (kernel_size - 1) * dilation
x_pad = F.pad(x, (__padding, 0))

In [11]:
torch.dot(convlayer.weight[0][0],x_pad[0][0:3])+convlayer.bias[0]

tensor(-0.0975, grad_fn=<AddBackward0>)

In [12]:
class context_embedding(torch.nn.Module):
    def __init__(self,in_channels=1,embedding_size=256,k=5):
        super(context_embedding,self).__init__()
        self.causal_convolution = CausalConv1d(in_channels,embedding_size,kernel_size=k)

    def forward(self,x):
        x = self.causal_convolution(x)
        return F.sigmoid(x)

In [13]:
embedding = context_embedding(1,256,5)

In [17]:
embedding(x.unsqueeze(0)).shape

torch.Size([1, 256, 20])